In [4]:
from keras import Model
from keras.models import Sequential, Layer
from keras.layers import Dense, MaxPool2D, Dropout, Activation, Input
from keras.optimizers import SGD, Adam, RMSprop
from keras import callbacks
from keras.layers import Flatten, LSTM
import numpy as np
import cv2
import numpy as np
import time
import queue
import copy
import math
import csv

Using TensorFlow backend.
/Users/shreyashkawalkar/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#input::: [disp,slp]

In [3]:
def paramotion(que):
    
    setpara = []
    pos1 = que.get()
    kk=9
    
    while(kk):
        pos2 = que.get()
        y = pos2[1] - pos1[1]
        x = pos2[0] - pos1[0]
        disp = round(y**2 + x**2,1)
        slp = round(math.atan2(y,x),2)
        setpara.append([disp, slp])
        pos1 = pos2
        kk-=1
    
    return setpara

In [21]:
import cv2
import numpy as np
import time
import queue
import copy
import math
import csv


vrec = cv2.VideoCapture('720p.mp4',0)
shape = [480,680]


setpara1 = []
setpara2 = []
datalist1 = []
datalist2 = []
labellist1 = []
labellist2 = []
pos11 = [0,0]
pos12 = [0,0]
pos21 = [0,0]
pos22 = [0,0]

kkk = 20
while(kkk):
    kkk-=1
    ret, scr1 = vrec.read()
    ret , scr2 = vrec.read()       

while True:
    start_time = time.time()
    ret , scr1 = vrec.read()
    ret, scr2 = vrec.read()
    if not ret:
        break
    scr1 = cv2.resize(scr1, (680,480))
    
    hsv = cv2.cvtColor(scr1,cv2.COLOR_BGR2HSV)
    hsv = cv2.inRange(hsv,(0, 70, 50),(180,255,255))
    
    cv2.imshow('scr1',scr1)

    im3, contours, newhierarchy = cv2.findContours(hsv, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    M = cv2.moments(contours[0])
    if(M['m00']==0):
        continue
    cx = float(M['m10']/M['m00'])
    cy = float(M['m01']/M['m00'])
#     print(cx,cy)
    pos12 = [cx,cy]
    if len(setpara1) == 9:
        datalist1+=[setpara1]
        labellist1+=[setpara1[-1]]
#         print("setpara1: ",setpara1)
#         print("lis1: ",labellist1)
        setpara1 = setpara1[1:]
    y = pos12[1] - pos11[1]
    x = pos12[0] - pos11[0]
    disp = round(y**2 + x**2,1)
    slp = round(math.atan2(y,x),2)
    setpara1.append([disp, slp])
    pos11 = pos12

    scr2 = cv2.resize(scr2, (680,480))
    hsv1 = cv2.cvtColor(scr2,cv2.COLOR_BGR2HSV)
    hsv1 = cv2.inRange(hsv1,(0, 70, 50),(180,255,255))
    
    cv2.imshow('scr2',scr2)

    im2, ncontours, hierarchy = cv2.findContours(hsv1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    M = cv2.moments(ncontours[0])
    if(M['m00']==0):
        continue
    cx = float(M['m10']/M['m00'])
    cy = float(M['m01']/M['m00'])
    pos22 = [cx,cy]
    if len(setpara2) == 9:
        datalist2+=[setpara2]
        labellist2+=[setpara2[-1]]
        setpara2 = setpara2[1:]
    y = pos22[1] - pos21[1]
    x = pos22[0] - pos21[0]
    disp = round(y**2 + x**2,1)
    slp = round(math.atan2(y,x),2)
    setpara2.append([disp, slp])
    pos21 = pos22
#         print("setpara2: ",setpara2)
#     print("FPS: ", 1.0 / (time.time() - start_time))
    q=cv2.waitKey(1)

    if q==27:
        cv2.destroyAllWindows()
        break;
#     break
cv2.destroyAllWindows()

In [22]:
datalist1 = datalist1[:-2]
datalist2 = datalist2[:-2]
labellist1 = labellist1[2:]
labellist2 = labellist2[2:]

In [23]:
import csv

with open('pathInertiaData.csv', 'w') as writeFile1:
    writer = csv.writer(writeFile1)
    writer.writerows(datalist1)
    writer.writerows(datalist2)
writeFile1.close()

with open('pathInertiaLabel.csv', 'w') as writeFile2:
    writer = csv.writer(writeFile2)
    writer.writerows(labellist1)
    writer.writerows(labellist2)
writeFile2.close()

In [16]:
len(data[0])

NameError: name 'data' is not defined

In [1]:
import csv

with open('pathInertiaData.csv', 'r') as readFile1:
    reader = csv.reader(readFile1)
    data = list(reader)
readFile1.close()

with open('pathInertiaLabel.csv', 'r') as readFile2:
    reader = csv.reader(readFile2)
    label = list(reader)
readFile2.close()

In [2]:
for ind in range(len(data)):
    data[ind] = list(map(eval,data[ind])) 

In [5]:
train_data = np.vstack(data)
train_target = np.array(label)
train_data.shape, train_target.shape
train_data = train_data.reshape((-1, 9, 2))
train_target = train_target.reshape((-1, 2))
train_data.shape, train_target.shape

((17517, 9, 2), (17517, 2))

In [8]:
model = Sequential()  
model.add(LSTM(2, input_shape=(9, 2), unroll=False, return_sequences=False))
model.add(Dense(2, activation='relu'))
model.compile(loss='mean_absolute_error', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 2)                 40        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 6         
Total params: 46
Trainable params: 46
Non-trainable params: 0
_________________________________________________________________


In [14]:
checkPoint = callbacks.ModelCheckpoint(filepath='pathInertia.h5', verbose=1, save_best_only=True)
res = model.fit(train_data, train_target, nb_epoch=200, batch_size=1, verbose=1,callbacks=[checkPoint],validation_split=0.2, shuffle=True)

Train on 14013 samples, validate on 3504 samples
Epoch 1/200
   15/14013 [..............................] - ETA: 2:44 - loss: 9.3873 

/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


14011/14013 [============================>.] - ETA: 0s - loss: 5.6190
Epoch 00001: val_loss improved from inf to 9.61692, saving model to pathInertia.h5
14013/14013 [==============================] - 127s 9ms/step - loss: 5.6201 - val_loss: 9.6169
Epoch 2/200
14009/14013 [============================>.] - ETA: 0s - loss: 5.6320
Epoch 00002: val_loss improved from 9.61692 to 6.33344, saving model to pathInertia.h5
14013/14013 [==============================] - 124s 9ms/step - loss: 5.6306 - val_loss: 6.3334
Epoch 3/200
14009/14013 [============================>.] - ETA: 0s - loss: 5.2969
Epoch 00003: val_loss did not improve
14013/14013 [==============================] - 125s 9ms/step - loss: 5.2965 - val_loss: 6.7354
Epoch 4/200
14009/14013 [============================>.] - ETA: 0s - loss: 6.9830
Epoch 00004: val_loss improved from 6.33344 to 6.22909, saving model to pathInertia.h5
14013/14013 [==============================] - 122s 9ms/step - loss: 6.9901 - val_loss: 6.2291
Epoch 5/2

KeyboardInterrupt: 

In [10]:
model_json = model.to_json()
with open("pathInertia.json", "w") as json_file:
    json_file.write(model_json)

In [24]:
with open("pathInertia.json", "r") as json_file:
    loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("pathInertia.h5")
    print("Model loaded from disk")
    loaded_model.summary()

Model loaded from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 2)                 40        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 6         
Total params: 46
Trainable params: 46
Non-trainable params: 0
_________________________________________________________________


In [29]:
loaded_model.predict([[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]])

AttributeError: 'list' object has no attribute 'ndim'

In [30]:
train_data

array([[[  7.92840000e+03,   5.00000000e-02],
        [  3.00000000e-01,   4.40000000e-01],
        [  1.00000000e-01,   2.30000000e-01],
        ..., 
        [  1.00000000e-01,   5.90000000e-01],
        [  4.00000000e-01,   3.10000000e-01],
        [  1.00000000e-01,   1.80000000e-01]],

       [[  3.00000000e-01,   4.40000000e-01],
        [  1.00000000e-01,   2.30000000e-01],
        [  4.00000000e-01,   2.30000000e-01],
        ..., 
        [  4.00000000e-01,   3.10000000e-01],
        [  1.00000000e-01,   1.80000000e-01],
        [  3.00000000e-01,   9.00000000e-02]],

       [[  1.00000000e-01,   2.30000000e-01],
        [  4.00000000e-01,   2.30000000e-01],
        [  1.00000000e-01,   3.30000000e-01],
        ..., 
        [  1.00000000e-01,   1.80000000e-01],
        [  3.00000000e-01,   9.00000000e-02],
        [  3.00000000e-01,   3.90000000e-01]],

       ..., 
       [[  0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00],
        [  0.000000